# Init

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

In [3]:
import sys

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.ser as ser

In [5]:
cmap='jet'

In [6]:
base_path = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15"
dic_dir = {"190130" : ["absC"],
           "190131" : ["absAHP12", "absB", "absD", "absJ"],
           "190201" : ["absA", "absAHP20", "absE", "absG", "absM"],
           "190204" : ["absAHP9", "absAHP30", "absAHP45", "absH", "absI", "absK", "absL", "absN", "absO", "absU"]}

In [7]:
nbAbs = 0
for key in dic_dir:
    nbAbs = nbAbs + len(dic_dir[key])
print( f"nbAbs = {nbAbs}" )

nbAbs = 20


# Relecture des données

In [9]:
dir_fondDeChambre = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15"
tmp = np.loadtxt(f"{base_path}/S11_fond_de_chambre.txt", skiprows=1)
S11_fondDeChambre = tmp[:,1] + 1j * tmp[:,2]

In [10]:
fig, ax = plt.subplots(1,1)
plt.plot( 20 * np.log10( np.abs( np.fft.ifft( S11_fondDeChambre ) ) ), label="fond de chambre" )
plt.grid()
plt.legend()

# Calcul du fond de chambre
ref: ser4siepel_multiband_scan_elev_F2C.sce

In [11]:
nb_elev = 41
nb_freq = 2001
elevation = np.linspace( 0.76 - 1.0, 0.76 + 1.0, nb_elev )

tab_S11_moy      = np.zeros( (nbAbs, nb_freq),   dtype=complex )
tab_S11_moy_td   = np.zeros( (nbAbs, nb_freq),   dtype=complex )
tab_S11_moy_tg   = np.zeros( (nbAbs, nb_freq),   dtype=complex )

data_S11         = np.zeros( (nb_elev, nb_freq), dtype=complex )
data_S11_td      = np.zeros( (nb_elev, nb_freq), dtype=complex )
data_S11_tg      = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy    = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy_td = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy_tg = np.zeros( (nb_elev, nb_freq), dtype=complex )
reflectivite     = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_moy          = np.zeros( nb_freq,            dtype=complex )
S11_moy_square   = np.zeros( nb_freq,            dtype=complex )

In [12]:
i_moy = 0
for key in dic_dir:
    print(key)
    for absx in dic_dir[key]:
        print(absx)
        seq = 0
        data_dir = f"{base_path}/{key}/{absx}" 
        S11_moy.fill( 0 )
        S11_moy_square.fill( 0 )
        
        for ielev in range(0, nb_elev):
            freq_all, S11_all = ser.getFreqAndS11( data_dir, seq0=seq, verbose=0 )
            seq = seq + 3
            S11_moy  = S11_moy + S11_all
            S11_moy_square = S11_moy_square + S11_all * np.abs( S11_all )
            d = np.linspace( 0, 300 / ( freq_all[1] - freq_all[0] ), len(freq_all) ) * 0.5
            gate = 0 * d + 1e-6;
            gate[ np.where( np.abs( d - 6650 ) < 500 ) ] = 1.0
            tab_freq = freq_all
        
        S11_moy        = S11_moy / nb_elev
        S11_moy_square = S11_moy_square / nb_elev
        S11_moy_td     = np.fft.ifft( S11_moy )
        S11_moy_tg     = S11_moy_td * gate

        tab_S11_moy[ i_moy,:]    = S11_moy
        tab_S11_moy_td[ i_moy,:] = S11_moy_td
        tab_S11_moy_tg[ i_moy,:] = S11_moy_tg
        
        i_moy = i_moy + 1
        
S11_moy    = np.mean( tab_S11_moy,    axis=0 )
S11_moy_td = np.mean( tab_S11_moy_td, axis=0 )
S11_moy_tg = np.mean( tab_S11_moy_tg, axis=0 )

190130
absC
190131
absAHP12
absB
absD
absJ
190201
absA
absAHP20
absE
absG
absM
190204
absAHP9
absAHP30
absAHP45
absH
absI
absK
absL
absN
absO
absU


In [13]:
fig, ax = plt.subplots( 1, 1 )
ax.plot( 20 * np.log10( np.abs( S11_fondDeChambre ) ), label='S11_fondDeChambre' )
ax.plot( 20 * np.log10( np.abs( S11_moy ) ), label='S11_moy' )
ax.grid()
ax.legend()

In [14]:
tmp[:,1] = np.real( S11_moy )
tmp[:,2] = np.imag( S11_moy )
np.savetxt( "S11_fond_de_chambre.txt", tmp, header="#GHz real imag")